In [ ]:
import laser_lib
import numpy as np
import math
import time
import cv2
import pyaudio  
import wave  
import threading
import time

In [ ]:
# Define the queue runner class
class QueueRunner:
    def __init__(self, laser_queue):
        self.cur_pattern = None
        self.laser_queue = laser_queue
        self.stopped = True
        # Start the polling thread
        poll_thread = threading.Thread(target=self.run_queue)
        poll_thread.start()
        
    def run_queue(self):
        while(True):
            if(self.cur_pattern is not None):
                self.stopped = False
                #print('Running queue...', end='\r')
                for arr_pos, arr_col in self.cur_pattern:
                    queue.submit(arr_pos, arr_col)
                    if(self.cur_pattern is None):
                        break
            else:
                #print('Queue Stopped...', end='\r')
                self.stopped = True
    
    def set_pattern(self, pattern):
        # Stop the pattern before setting a new one
        self.stop_pattern()
        # Set the new pattern
        self.cur_pattern = pattern
        # Don't return until the pattern has started
        # TODO - maybe we don't have to make this blocking
        while(self.stopped):
            continue
    
    def stop_pattern(self):
        self.cur_pattern = None
        # Wait until the thread acknowledges the stop
        # TODO - maybe we don't have to make this blocking
        while(not self.stopped):
            continue
    

def p1(T):
    theta = np.linspace(0, 2*np.pi, T)
    arr_pos = np.zeros((T, 2))
    arr_col = np.ones((T, 3))
    arr_pos[:, 0] = np.sin(theta)
    arr_pos[:, 1] = np.cos(theta)
    while(True):
        yield arr_pos, arr_col
    
    
            
        

In [ ]:
queue = laser_lib.DacQueue()
runner = QueueRunner(queue)

In [ ]:
#define stream chunk size
chunk = 512  
#open a wav format music  
f = wave.open(r"Villain.wav","rb")  
# Set to further into the file
f.setpos(0)
#instantiate PyAudio  
p = pyaudio.PyAudio()  
#open stream  
stream = p.open(format = p.get_format_from_width(f.getsampwidth()),  
                channels = f.getnchannels(),  
                rate = f.getframerate(),  
                output = True)  
#read data  A
data = f.readframes(chunk)  

# list of patterns of type {'start_time', 'end_time', pattern}
sequence_queue = []
# make some random patterns
#for i in range(0, 100, 3):
#    start = np.random.rand(1)*i
#    stop = start + 1
#    pattern_queue += [{'start_time': start, 'stop_time': stop, 'sequence': p1}]
#sequence_queue += [{'start_time': 0, 'stop_time': 1, 'sequence': p1}]
#sequence_queue += [{'start_time': 3, 'stop_time': 5, 'sequence': p1}]
#sequence_queue += [{'start_time': 10, 'stop_time': 14, 'sequence': p1}]
#sequence_queue += [{'start_time': 20, 'stop_time': 30, 'sequence': p1}]
song_len = 60*3
for i in range(0, 180, 1):
    sequence_queue += [{'start_time': i, 'stop_time': i+0.5, 'sequence': p1}]


# Define the start/stop times of patterns
times_arr = np.zeros((len(sequence_queue), 2))
for idx, seq in enumerate(sequence_queue):
    times_arr[idx, :] = np.array([seq['start_time'], seq['stop_time']])
# Tracks which sequences have been run already
ran_arr = np.zeros((len(sequence_queue))).astype(np.bool)

sample_count = 0

all_idxs = np.arange(len(sequence_queue))
framerate = f.getframerate()
while(data):  
    # Write to soundcard
    stream.write(data)  
    # Grab the next chunk of music
    data = f.readframes(chunk)  
    cur_time = sample_count/framerate
    # Find a pattern to display
    
    # Find out if a pattern should be playing
    valid_patterns = np.logical_and(cur_time >= times_arr[:, 0], cur_time < times_arr[:, 1])
    valid_idxs = all_idxs[valid_patterns]
    if(len(valid_idxs) > 0):
        # Run the pattern if it hasn't been run before
        if(not ran_arr[valid_idxs[0]]):
            #print("val idxs", valid_idxs)
            runner.set_pattern(sequence_queue[valid_idxs[0]]['sequence'](500))
            ran_arr[valid_idxs[0]] = True
    else:
        runner.stop_pattern()

    print(cur_time, end='\r')
    sample_count += chunk
    
    # Turn into numpy array
    #data_np = np.frombuffer(data, dtype=np.int16).astype(np.float32)
    # Reshape into two channels
    #data_np = np.reshape(data_np, (-1, 2))

#stop stream  
stream.stop_stream()  
stream.close()  

#close PyAudio  
p.terminate()  

In [ ]:
len(sequence_queue)

In [ ]:
f.getframerate()/chunk

In [ ]:
runner = QueueRunner(queue)

In [ ]:
runner.set_pattern(p1(500))

In [ ]:
runner.stop_pattern()

In [ ]:
for arr_pos, arr_col in p1(500):
#arr_pos, arr_col = next(p1(500))
#while(True):
    queue.submit(arr_pos, arr_col)    

In [ ]:
for i in range(1000):
    time.sleep(0.04)
    runner.set_pattern(p1(500))
    time.sleep(0.04)
    runner.stop_pattern()

In [ ]:
music_data = np.zeros((T, 2))
t = 0
chunk_size = 1/2 sec

runner = Runner()
while(t < T):
    data = music_data[t:t+chunk_size]

    play(data)
    
    # non-blocking call
    runner.changepattern()
    
    t += chunk_size